# Rec Well User data from Fall 2022 to Summer 2024

Rec Well, the organization that manages the recreational facilities such as the Nick and Bakke, has a wealth of data from their database. With the completion of Bakke in April 2023, Rec Well wants to see how users are using the Nick and Bakke facilities. 

* How many students are they serving?
* How many different users participate in different programs?
* Which building does a certain group go to?

These are the questions that we will want to answer. Their database (called Fusion) stores all this data, but we need to visualize the data to get a proper grasp of how different numbers compare with each other.



# Data Background
Users that visit Rec Well are often students, but they also service a wider community. Users that they have categorized in their database are separated into different groups, such as Community Members, Graduate Students, Campus Affiliates, Day/Week User, and more. From these different user types, we want to figure out several different things from them. These can include:

* Facilities they go to
* Programs they enroll in (yoga, ice skating lessons, cooking lessons, etc.)
* The date range of their membership date

The data that we have recieved from Rec Well contains all the facility accesses. They have also given us files for each program, but can contain inconsistencies on how the data is structured. In the end, we want to see a user and how that individual has interacted with the facilities and programs. In short, we will want a master file where each row will contain 

* a user ID
* their membership type
* their active membership date
* number of check ins at Rec Well facilities
* and the number of participations in programs

In [ ]:
import pandas as pd

# Question 1

First we'll work with the facility access data. Their data begins in Fall 2022. Each semester is divided into its own file up until Summer 2024. Everytime a person scans in at the entrance of the recreational facility, their identification, membership, and time stamp are all recorded. Let's load the CSV's and examine how the data is structured.

In [ ]:
df_fall22 = pd.read_csv('access_fall22.csv')
df_fall23 = pd.read_csv('access_fall23.csv')
df_spring23 = pd.read_csv('access_spring23.csv')
df_spring24 = pd.read_csv('access_spring24.csv')
df_summer23 = pd.read_csv('access_summer23.csv')
df_summer24 = pd.read_csv('access_summer24.csv')


This shows us that the files contain identifiying information like an `ID_NUMBER`, `FIRST_NAME` and `LAST_NAME`, `EMAIL_ADDRESSES` and `EXTERNAL_ID_NUMBERS`. `ID_NUMBER` is how their database uniquely identifies their users. First off, we will prep the files to be merged all together. Lets look at the value counts for the facilities. 

In [ ]:
df_fall22['FACILITY_NAME'].value_counts()
df_spring24['FACILITY_NAME'].value_counts()

The `Nick - Fitness Tablets`, `Nick - Welcome Desk`, and `Nick - Wellbeing Services 124` should all be considered as a single facility called `Nick` in `df_fall22`. Likewise, `Bakke - Welcome Desk`, `Bakke - Fitness Tablets`, `Bakke - Member Services` and `Bakke - Wolf Teaching Kitchen` should all be named to simply `Bakke`. However, `Bakke - Sub-Zero Ice Center` and `Bakke - Mt. Mendota` should be kept separately as those entries can only be tracked by this facility access file. 

Create a function that will consolidate these the fields into `Nick` and `Bakke` respectively while not touching `Bakke - Sub-Zero Ice Center` and `Bakke - Mt. Mendota` for each dataframe.

In [ ]:
def locations_processed(df, column_name):
    unchanged_values = ['Bakke - Mt. Mendota', 'Bakke - Sub-Zero Ice Center', 'Nielsen Tennis Stadium', 'Camp Randall Sports Center (The SHELL)']
    df[column_name] = df[column_name].apply(lambda x: 'Bakke' if 'Bakke' in x and x not in unchanged_values 
                                            else ('Nick' if 'Nick' in x or 'Nicholas' in x else x))
    return df

df_fall22 = locations_processed(df_fall22, 'FACILITY_NAME')
df_fall23 = locations_processed(df_fall23, 'FACILITY_NAME')
df_spring23 = locations_processed(df_spring23, 'FACILITY_NAME')
df_spring24= locations_processed(df_spring24, 'FACILITY_NAME')
df_summer23= locations_processed(df_summer23, 'FACILITY_NAME')
df_summer24= locations_processed(df_summer24, 'FACILITY_NAME')

# Question 2

Let's test if the function properly worked. Save the `value_counts()` of facilities in`df_spring24` to a variable `locations_check`. The value for the `Nick` should be 514479 and the `Bakke` should be 340415

In [ ]:
locations_check = df_spring24['FACILITY_NAME'].value_counts()
locations_check

# Question 3

Next we will load the Membership file. This file contains every person that has entered in the facilities and what their membership is. This file was already consolidated through the SQL query used to generate it. Let's load it, then view the different types of memberships. Also, note that there are `Start_Date` and `End_Date` columns in the dataset, in contrast to the facility access file's `CREATION_TIMESTAMP`

In [ ]:
df_member=pd.read_csv('all_memberships.csv')


For each of our facility access dataframes, we will be compiling them and adding them to the membership file. This means that if they have entered the Bakke 50 times in a semester (which means 50 rows in the facility access dataframes), then there will be a tally of 50. We also need to take into account of the date. If the user was a part time student but dropped mid-way in a semester and then became a community member, then we need to take their membership dates into account. 

We will create a function that will perform this action for all the dataframes. 


The first step we will do is handling the dates. Create a function to check if the `CREATION_TIMESTAMP` lies between one of the user's `Start_Date` and `End_Date` (convert to datetime to compare) after performing an inner merge from the membership dataframe to the access dataframe to gather the accesses that are valid. This will only be the initial step of the function and we will be adding more onto it.

*Note*: entries in the `End_Date` column can be empty. This is because they can still be active users, so they do not have an end date! We must accomodate for this when comparing the dates. Since they are still currently active users, we will set their date to be a very far year in the future, and will revert it later. For example, it can look like this:`pd.Timestamp('2262-04-11')`


In [ ]:
def merging_df(access_df, memberships_df): 
    memberships_df['Start_Date'] = pd.to_datetime(memberships_df['Start_Date'], format='mixed')
    memberships_df['End_Date'] = pd.to_datetime(memberships_df['End_Date'], format='mixed')
    access_df['CREATION_TIMESTAMP'] = pd.to_datetime(access_df['CREATION_TIMESTAMP'], format='mixed')
    memberships_df['End_Date']=memberships_df['End_Date'].fillna(pd.Timestamp('2262-04-11'))

    merged_df = access_df.merge(memberships_df, on='ID_NUMBER', how='inner')
    
    valid_access = merged_df[(merged_df['CREATION_TIMESTAMP']>= merged_df['Start_Date']) &
        (merged_df['CREATION_TIMESTAMP']<= merged_df['End_Date'])]
    return valid_access

merging_df(df_fall22, df_member)
    

# Question 4

Now we will be counting accesses per membership period and facility. We will have to use the `groupby()` function to group the data and count it. After we generate a count, we need to pivot the table so that the column names will be the `FACILITY_NAME` and our index will be `ID_Number`, `Start_Date`, `End_Date`, and `Membership L2`.

After this, we will be merging the pivoted dataframe onto the `membership_df` on `ID_NUMBER`, `Start_Date`, `End_Date`, `Membership_L2` columns. This will generate new columns in the dataframe, so we should distinguish the new columns with something (example: `'_new'`)

We will be adding onto the previous function since we are performing merges and do not want to replicate merges.

In [ ]:
def merging_df(access_df, memberships_df, access_count_col='Access_Count'):
    memberships_df['Start_Date'] = pd.to_datetime(memberships_df['Start_Date'],format='mixed')
    memberships_df['End_Date'] = pd.to_datetime(memberships_df['End_Date'], format='mixed')
    access_df['CREATION_TIMESTAMP'] = pd.to_datetime(access_df['CREATION_TIMESTAMP'], format='mixed')
    memberships_df['End_Date']=memberships_df['End_Date'].fillna(pd.Timestamp('2262-04-11'))

    merged_df = access_df.merge(memberships_df, on='ID_NUMBER', how='inner')
    
    valid_access = merged_df[(merged_df['CREATION_TIMESTAMP']>= merged_df['Start_Date']) &
        (merged_df['CREATION_TIMESTAMP']<= merged_df['End_Date'])]
    
##########################
    
    access_counts = valid_access.groupby(['ID_NUMBER','Start_Date', 'End_Date', 'Membership_L2', 'FACILITY_NAME']).size().reset_index(name=access_count_col)
    pivoted_access_counts = access_counts.pivot_table(
        index = ['ID_NUMBER', 'Start_Date', 'End_Date', 'Membership_L2'],
        columns = 'FACILITY_NAME',
        values=access_count_col,
        fill_value=0).add_suffix("_new").reset_index()
    
    merged_access_counts=memberships_df.merge(pivoted_access_counts,
                                              on=['ID_NUMBER', 'Start_Date', 'End_Date', 'Membership_L2'],
                                              how='left').fillna(0)
    return merged_access_counts
                                              
merging_df(df_fall22, df_member)



# Question 5

Now we will be finishing the merge. For each facility name, we want a column to be created in the final merged df and for them to be integers. If the column already exists, we want to columns to be added. If it does not exist, we want the column to be created and add the sum of the facility access into the column. We also want to drop the new columns(the ones with the suffix `_new`) so that it will make it easier to read and properly sum our totals.

In addition, we will be reverting the `End_Date` back to the be `NaT` for the ones that we changed to be `2262-04-11`. Use the function to combine all the access files (eg. `df_fall22`) into the membership file (`df_member`) sequentially and save them in a vairable `new_df`

In [ ]:
def merging_df(access_df, memberships_df, access_count_col='Access_Count'):
    memberships_df['Start_Date'] = pd.to_datetime(memberships_df['Start_Date'], format='mixed')
    memberships_df['End_Date'] = pd.to_datetime(memberships_df['End_Date'], format='mixed')
    access_df['CREATION_TIMESTAMP'] = pd.to_datetime(access_df['CREATION_TIMESTAMP'], format='mixed')
    memberships_df['End_Date']=memberships_df['End_Date'].fillna(pd.Timestamp('2262-04-11'))

    merged_df = access_df.merge(memberships_df, on='ID_NUMBER', how='inner')
    
    valid_access = merged_df[(merged_df['CREATION_TIMESTAMP']>= merged_df['Start_Date']) &
        (merged_df['CREATION_TIMESTAMP']<= merged_df['End_Date'])]
    
##########################
    
    access_counts = valid_access.groupby(['ID_NUMBER','Start_Date', 'End_Date', 'Membership_L2', 'FACILITY_NAME']).size().reset_index(name=access_count_col)
    pivoted_access_counts = access_counts.pivot_table(
        index = ['ID_NUMBER', 'Start_Date', 'End_Date', 'Membership_L2'],
        columns = 'FACILITY_NAME',
        values=access_count_col,
        fill_value=0).add_suffix("_new").reset_index()
    
    merged_access_counts = memberships_df.merge(pivoted_access_counts,
                                              on=['ID_NUMBER', 'Start_Date', 'End_Date', 'Membership_L2'],
                                              how='left').fillna(0)

##############################
                                            
    facility_names = access_df['FACILITY_NAME'].unique()     

    for col in facility_names:
        if col in merged_access_counts.columns:
            merged_access_counts[col] += merged_access_counts[f"{col}_new"]
        else:

            merged_access_counts[col] = merged_access_counts[f"{col}_new"]
        merged_access_counts.drop(columns = [f"{col}_new"], inplace=True)
            
    merged_access_counts['End_Date'] = merged_access_counts['End_Date'].replace(pd.Timestamp('2262-04-11'), pd.NaT)

    merged_access_counts[facility_names]=merged_access_counts[facility_names].astype(int)
    return merged_access_counts

new_df=merging_df(df_fall22, df_member)
new_df=merging_df(df_spring23, new_df)
new_df=merging_df(df_summer23, new_df)
new_df=merging_df(df_fall23, new_df)
new_df=merging_df(df_spring24, new_df)
new_df=merging_df(df_summer24, new_df)
new_df



# Question 6

Next we will be loading the programs into the kernel and be merging these files with the file that we have just created. This file is has several sheets for each different semester so we'll be reading the csv by sheet. The end result will look similar to what we have created, a tally of all the times that a user has participated in a certain program or activity. Load all the sheets for each semester (fall 2022, spring 2023, summer 2023, fall 2023, spring 2024, summer 2024) and rename the columns `Id Number` to be `ID_NUMBER` and `Date/Time Registered` to `Date Registered` for consistence and clarity. In addition, we'll load in a csv file that contains all the programs to include in this merged dataframe

In [ ]:
programs_fall22= pd.read_csv("Fusion Fall 2022.csv")
programs_spring23= pd.read_csv("Fusion Spring 2023.csv")
programs_summer23=pd.read_excel("Data - Fusion Program Registration (Summer 23 - Summer 24).xlsx", sheet_name='Summer 23')
programs_fall23=pd.read_excel("Data - Fusion Program Registration (Summer 23 - Summer 24).xlsx", sheet_name='Fall 23')
programs_spring24=pd.read_excel("Data - Fusion Program Registration (Summer 23 - Summer 24).xlsx", sheet_name='Spring 24')
programs_summer24=pd.read_excel("Data - Fusion Program Registration (Summer 23 - Summer 24).xlsx", sheet_name='Summer 24')

df_program_list = pd.read_csv("List of all programs_2024.11.12.csv")

programs_fall22.rename(columns={'Id Number': "ID_NUMBER", 'Date/Time Registered': 'Date Registered'}, inplace =True)
programs_fall23.rename(columns={'Id Number': "ID_NUMBER"}, inplace =True)
programs_spring23.rename(columns={'Id Number': "ID_NUMBER", 'Date/Time Registered': 'Date Registered'}, inplace =True)
programs_spring24.rename(columns={'Id Number': "ID_NUMBER"}, inplace =True)
programs_summer23.rename(columns={'Id Number': "ID_NUMBER"}, inplace =True)
programs_summer24.rename(columns={'Id Number': "ID_NUMBER"}, inplace =True)


# Question 7

Lets make our `program_list` and actual `list` of program names to use and filter off of. Due to how `Youth Programs` work and how they're registered, we wont be able to use that data. We'll have to exclude them out of our data set. Make a list without any of the youth programs and save it to a vairbale named `list_of_programs`

In [ ]:
list_of_programs=df_program_list['Category Name'].unique().tolist()
list_of_programs=[x for x in list_of_programs if 'Youth' not in x]
list_of_programs

# Question 8
Now, we will make a function to clean our program files (eg `programs_fall22`). We will drop the columns that are listed as exclude in the`Include/Exclude` column in the `programs_list` file. This will be a merge and will be on `Program Name`. Remember to filter the dataframe with `list_of_programs` since the `Youth` programs are marked to `Include` (cannot use the data due to how registration works and the ambiguity of their membership).

In [ ]:
def cleaning_programs(program_list_df, access_df):
    program_list_df.columns = program_list_df.columns.str.strip()
    access_df.columns = access_df.columns.str.strip()

    merged_df = access_df.merge(
        program_list_df[['Program Name', 'Category Name', 'Include/Exclude']],
        on='Program Name',
        how='left')

    merged_df['Category Name_x'] = merged_df['Category Name_y'].fillna(merged_df['Category Name_x'])
    merged_df = merged_df.rename(columns = {'Category Name_x': 'Category Name'})
    merged_df.drop(columns = ['Category Name_y'], inplace=True)

    cleaned_df = merged_df[merged_df['Include/Exclude'] != 'Exclude'].drop(columns=['Include/Exclude'])
    cleaned_df = cleaned_df[cleaned_df['Category Name'].isin(list_of_programs)]
##################################################
    # keep_columns = ['ID_NUMBER', 'Category Name', 'Program Name', 'Date Registered']
    
    return cleaned_df

programs_fall22_cleaned=cleaning_programs(df_program_list, programs_fall22)
programs_spring23_cleaned=cleaning_programs(df_program_list, programs_spring23)
programs_summer23_cleaned=cleaning_programs(df_program_list, programs_summer23)
programs_fall23_cleaned=cleaning_programs(df_program_list, programs_fall23)
programs_spring24_cleaned=cleaning_programs(df_program_list, programs_spring24)
programs_summer24_cleaned=cleaning_programs(df_program_list, programs_summer24)






# Question 9

We will now be doing very similar data transformations as we did in question 5. 

We will make user all our dates are in datetime format for comparison

We will verify the registration of the date of program within the user's active membership date (Handle the `End_Date` the same as before)

We will perform an inner merge from the membership dataframe onto the programs dataframe to find all valid users on `ID_NUMBER`

We will groupby `ID_NUMBER`, `Start_Date`, `End_Date`, `Membership_L2`, and `Category Name`. This will then be pivoted again with our columns being `Category Name`

We will left merge this pivoted dataframe back onto the membership file on `ID_NUMBER`, `Start_Date`, `End_Date`, and `Membership_L2`.

Then drop unnecessary columns created after merging (Dont forget to change the `End_Date` back to being a `NaT`)



In [ ]:
def registration_counts_by_program(program_df, memberships_df, access_count_col='Access Count'):
    memberships_df['Start_Date'] = pd.to_datetime(memberships_df['Start_Date'], format='mixed')
    memberships_df['End_Date'] = pd.to_datetime(memberships_df['End_Date'], format='mixed')
    program_df['Date Registered'] = pd.to_datetime(program_df['Date Registered'], format='mixed', errors='coerce')
    memberships_df['End_Date']=memberships_df['End_Date'].fillna(pd.Timestamp('2262-04-11'))
    
    program_df.columns=program_df.columns.str.strip()
    merged_df = program_df.merge(memberships_df, on='ID_NUMBER', how='inner')
    
    
    valid_access = merged_df[(merged_df['Date Registered']>= merged_df['Start_Date']) &
        (merged_df['Date Registered']<= merged_df['End_Date'])]
    
##########################
    
    access_counts = valid_access.groupby(['ID_NUMBER','Start_Date', 'End_Date', 'Membership_L2', 'Category Name']).size().reset_index(name=access_count_col)
    
    pivoted_access_counts = access_counts.pivot_table(
        index = ['ID_NUMBER', 'Start_Date', 'End_Date', 'Membership_L2'],
        columns = 'Category Name',
        values=access_count_col,
        fill_value=0).add_suffix("_new").reset_index()
    
    merged_access_counts = memberships_df.merge(pivoted_access_counts,
                                              on=['ID_NUMBER', 'Start_Date', 'End_Date', 'Membership_L2'],
                                              how='left').fillna(0)

 
    
    program_names = program_df['Category Name'].unique()
    # program_names = [name.strip() for name in program_names] 
    
    for col in program_names:
        if col in merged_access_counts.columns:
            merged_access_counts[col] += merged_access_counts[f"{col}_new"]
        else:

            merged_access_counts[col] = merged_access_counts[f"{col}_new"]
            
            
            
        merged_access_counts.drop(columns = [f"{col}_new"], inplace=True)
            
    merged_access_counts['End_Date'] = merged_access_counts['End_Date'].replace(pd.Timestamp('2262-04-11'), pd.NaT)

    merged_access_counts[program_names]=merged_access_counts[program_names].astype(int)
    return merged_access_counts

final_df = registration_counts_by_program(programs_fall22_cleaned, new_df)
final_df = registration_counts_by_program(programs_spring23_cleaned, final_df)
final_df = registration_counts_by_program(programs_summer23_cleaned, final_df)
final_df = registration_counts_by_program(programs_fall23_cleaned, final_df)
final_df = registration_counts_by_program(programs_spring24_cleaned, final_df)
final_df = registration_counts_by_program(programs_summer24_cleaned, final_df)
final_df

    

# Question 10

With our final dataframe created, we can now make visualizations based on the students and their participations in different programs. Before this, we should ensure that our columns are populated using `sum()`. Since many of the users do not participate in these programs, much of the dataframe will appear to be 0. Find the `sum()` of participants in `Outdoor Yoga` and find the number of unique participants in that program as well.

In [ ]:
total=final_df['Outdoor Yoga'].sum()
counts=len(final_df[final_df['Outdoor Yoga']>0])
print('Total: ', total, '\nUnique: ', counts)

# Question 11
Let's start visualizing the data and answering questions. Plot how many total users there are in each `Membership_L2` as a bar graph and label each bar. Make the `color` for each bar a different color to your choosing. Remember to label the x-axis and y-axis and give it the title `Membership User Totals`. In addition, there is a value called `Student` in this column. This is incorrectly labeled, so change all the values with `Student` with `Student-Other`.

In [ ]:
import matplotlib.pyplot as plt
def label(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], str(y[i]), ha='center', fontsize='small')

colors = ['k','b','g','r','y','lightblue','teal','pink']

final_df.loc[final_df['Membership_L2']=='Student', 'Membership_L2'] = 'Student-Other'
y_axis=final_df['Membership_L2'].value_counts()
x_axis=y_axis.index
ax=y_axis.plot.bar(color=colors)
label(x_axis,y_axis)
ax.set_xlabel('User Type')
ax.set_ylabel("Number of Users")
ax.set_title('Membership User Totals')


# Question 12
Now, lets view the data for the buildings. Create a plot that shows how many users enter the `Nick`, `Bakke`, and `Nielsen Tennis Stadium` buildings for the date range `09/01/2023` to `05/16/2024`. Add x and y-axis labels and title `Facility Access Fall 23 and Spring 24`.

In [ ]:

def label(x,y):
    for i in range(len(x)):
        ax.text(i, y[i]+400, str(y[i]), ha='center', fontsize='small')

colors = ['b','g','r','y','lightblue','teal','pink']

y_axis=final_df[(final_df['Start_Date'] >= '2023/09/01')&
(final_df['Start_Date'] <= '2024/05/16')&
(final_df['End_Date'] >= '2023/09/01')&
(final_df['End_Date'] <= '2024/05/16')][['Nick', 'Bakke', 'Nielsen Tennis Stadium']].sum()

ax=y_axis.plot.bar(color=colors)
label(y_axis.index ,y_axis)
ax.set_xlabel('Facility')
ax.set_ylabel("Number of Users")
ax.set_title('Facility Access Fall 23 and Spring 24')


In [ ]:
final_df.info()

# Question 13
Now lets look at the `Tennis Programs`. Rec Well wants to see how many `Community`  and `Undergrad` members they serve in their `Tennis Programs` from the date range `09/01/2023` to `05/16/2024`. Compare these two user types side by side.

In [ ]:

colors = ['b','g','r','y','lightblue','teal','pink']

community=final_df[(final_df['Start_Date'] >= '2023/09/01')&
(final_df['Start_Date'] <= '2024/05/16')&
(final_df['End_Date'] >= '2023/09/01')&
(final_df['End_Date'] <= '2024/05/16')&
(final_df['Membership_L2'] == 'Community')]

undergrad=final_df[(final_df['Start_Date'] >= '2023/09/01')&
(final_df['Start_Date'] <= '2024/05/16')&
(final_df['End_Date'] >= '2023/09/01')&
(final_df['End_Date'] <= '2024/05/16')&
(final_df['Membership_L2'] == 'Undergrad')]

total_tennis_community=community['Tennis Programs'].sum()
total_tennis_undergrad=undergrad['Tennis Programs'].sum()

summed_series = pd.Series([total_tennis_community, total_tennis_undergrad], index=['Community', 'Undergrad'])

ax = summed_series.plot.bar( color='green', width=0.3)

for i in range(len(summed_series)):
    ax.text(i, summed_series[i]+0.1, str(summed_series[i]), ha='center') 

ax.set_xlabel('User Type')
ax.set_ylabel('Participations')
ax.set_title('Tennis Program Participations for Community and Undergrad Users')
ax.tick_params(axis= 'x', rotation=0)

# Question 14

What is the percent of users that participate in `Nutrition` programs are `Grad` students? What percent of the users are `Undergrad` students? Plot these two percentages together.

In [ ]:
nutrition_grads=final_df[(final_df['Membership_L2'] == 'Grad')&(final_df['Nutrition'] >0)]['Nutrition']
nutrition_undergrads=final_df[(final_df['Membership_L2'] == 'Undergrad')&(final_df['Nutrition'] >0)]['Nutrition']
nutrition_totals = final_df[final_df['Nutrition'] >0]['Nutrition']

percent_grad=(nutrition_grads.sum()/nutrition_totals.sum())*100
percent_undergrad=(nutrition_undergrads.sum()/nutrition_totals.sum())*100

percentages=pd.Series([percent_grad, percent_undergrad], index=['Grad', 'Undergrad'])

ax=percentages.plot.bar(color='purple')

for i, percent in enumerate(percentages):
    ax.text(i, percent+0.5, f'{percent:.2f}%', ha='center')

ax.set_xlabel('User Type')
ax.set_ylabel('Percentage')
ax.set_title('Grad and Undergrad Participation Percentage in Nutrition Programs')
ax.tick_params(axis= 'x', rotation=0)

# Question 15

Rec Well wants to find out how many unique users participate in `Fitness: Group Classes Bakke` compared to `Fitness: Group Classes Nick`. Compare these two on a plot.

In [ ]:
nick_counts=len(final_df[final_df['Fitness: Group Classes Nick']>0])
bakke_counts=len(final_df[final_df['Fitness: Group Classes Bakke']>0])

total_counts = pd.Series([nick_counts, bakke_counts], index=['Group Fitness Nick', 'Group Fitness Bakke'])

ax=total_counts.plot.bar(color='pink')

for i in range(len(total_counts)):
    ax.text(i, total_counts[i]+60, str(total_counts[i]), ha='center', fontsize='small')


ax.set_xlabel('Class Type')
ax.set_ylabel('Participation Numbers')
ax.set_title('Group Fitness Class Participations at Nick and Bakke')
ax.tick_params(axis= 'x', rotation=0)

# Question 16

(Grouped bar graphs)
Rec Well wants to figure out how many unique users use both the `Group Fitness Nick` and `Group Fitness Bakke`. Use the visual from question 15 and to compare the three columns together.  

In [ ]:
nick_bakke_counts=len(final_df[(final_df['Fitness: Group Classes Nick']>0)&
                      (final_df['Fitness: Group Classes Bakke']>0)])
nick_counts=len(final_df[final_df['Fitness: Group Classes Nick']>0])
bakke_counts=len(final_df[final_df['Fitness: Group Classes Bakke']>0])

total_counts = pd.Series([nick_counts, bakke_counts, nick_bakke_counts], 
                         index=['Group Fitness Nick', 'Group Fitness Bakke', 'Dual Users'])

ax=total_counts.plot.bar(color='pink')

for i in range(len(total_counts)):
    ax.text(i, total_counts[i]+60, str(total_counts[i]), ha='center', fontsize='small')

ax.set_xlabel('Class Type')
ax.set_ylabel('Participation Numbers')
ax.set_title('Group Fitness Class Participations at Nick and Bakke')
ax.tick_params(axis= 'x', rotation=0)

# Question 17

(compare fall 2023 to fall 2024 program data)
Rec Well wants to see how many users started in `Fall 2022` and utilized the `Nick` and compare this number to how many started in `Fall 2023`. This number shows the amount of users starting their membership in the fall, which includes new students, recurring students, and any user that registers within the fall for their services. Then, as long as they entered the `Nick` to scan their Wiscard, this graph will capture their swipe. Remember that the date range for Fall 2022 is `08/24/2022` to `12/25/2022` and Fall 2023 is `08/24/2023` to `12/25/2024`. Why does the graph look like this? 

In [ ]:

fall_2022=final_df[(final_df['Start_Date'] >= '2022/08/24')&
(final_df['Start_Date'] <= '2022/12/25')]


fall_2023=final_df[(final_df['Start_Date'] >= '2023/08/24')&
(final_df['Start_Date'] <= '2023/12/25')]



totals_fall2022=len(fall_2022[fall_2022['Nick']>0])
totals_fall2023=len(fall_2023[fall_2023['Nick']>0])

total_counts = pd.Series([totals_fall2022, totals_fall2023], 
                         index=['Fall 2022', 'Fall 2023'])

ax=total_counts.plot.bar(color='brown')

for i in range(len(total_counts)):
    ax.text(i, total_counts[i]+50, str(total_counts[i]), ha='center', fontsize='small')

ax.set_xlabel('Semester')
ax.set_ylabel('Registration Numbers')
ax.set_title('New Users Registered and Entering the Nick Across Semesters')
ax.tick_params(axis= 'x', rotation=0)

# Question 18

Rec Well wants to check how the `Climbing` program data compares to the `Bakke - Mt. Mendota` field. the `Bakke - Mt. Mendota` is for users tht walk in and go to access the climbing wall while the `Climbing` program displays specific sessions in where users can register to climb with instructors. These two fields are exclusive of each other. If someone registers for the `Climbing` program and uses the wall, they are not counted for that instance of using the `Bakke - Mt. Mendota`. What is the percent of users that participate in the `Climbing` program that use the `Bakke - Mt. Mendota` wall and what percent is that of the `Bakke` population? Remember to get unique users and not the number of participations. 

The resulting graph shows how many users engaging in the `Climbing` Program and visit the `Bakke - Mt. Mendota` wall. This percentage can be indicative of how effective the program is at retaining users and bringing them back. Or it can also show engagement of regular users entering the programs and helping others try and enter the sport.


In [ ]:
mendota=len(final_df[final_df['Bakke - Mt. Mendota']>0]['Bakke - Mt. Mendota'])
climbing=len(final_df[final_df['Climbing']>0]['Climbing'])
bakke_total = len(final_df[final_df['Bakke'] >0]['Bakke'])

percent_mendota=(mendota/bakke_total)*100
percent_climbing=(climbing/bakke_total)*100
percent_climbing_of_mendota=(climbing/mendota)*100

colors = ['gold','gold', 'grey']

percentages=pd.Series([percent_mendota,percent_climbing, percent_climbing_of_mendota], 
                      index=['Bakke - Mt. Mendota', 'Climbing Program', '% of Climbing Program Users in Mt. Mendota' ])

ax=percentages.plot.bar(color=colors, figsize=(10,5))

for i, percent in enumerate(percentages):
    ax.text(i, percent+0.15, f'{percent:.3f}%', ha='center')

ax.set_xlabel('Program Type')
ax.set_ylabel('Percentage')
ax.set_title('Users in Climbing Programs and Mt. Mendota Percentage')
ax.tick_params(axis= 'x', rotation=0)

# Question 19

As a whole, view all the programs in a single graph and display the most popular graph in terms of participations and unique users. Do this as a grouped horizontal bar graph. 

In [ ]:
import numpy as np
columns=final_df.iloc[:, 18:]

category_unique = columns.apply(lambda x: (x > 0).sum())
fig, ax = plt.subplots(figsize=(11, 9))
column_sums=columns.sum()
n = len(category_unique)

y_pos = np.arange(n)

bar_height = 0.4

unique_bar=ax.barh(y_pos - bar_height / 2, category_unique, bar_height, color='lightblue', edgecolor='black', label='Unique Users')
participant_bar=ax.barh(y_pos + bar_height / 2, column_sums, bar_height, color='gold', edgecolor='black', label='Participations')

for bar in bars1:
    ax.text(bar.get_width() + 30, bar.get_y() + bar.get_height() / 2, 
            f'{bar.get_width():.0f}', va='center', ha='left', fontsize=9) 
for bar in bars2:
    ax.text(bar.get_width() + 30, bar.get_y() + bar.get_height() / 2,
            f'{bar.get_width():.0f}', va='center', ha='left', fontsize=9)  
ax.set_xlabel('Number of Users')
ax.set_ylabel('Program Name')
ax.set_title('Users in Climbing Programs and Mt. Mendota Percentage')
ax.set_yticks(y_pos)
ax.set_yticklabels(category_unique.index)
ax.legend()


# Question 20
`Fitness: Group Classes Nick` contains the most number of users. Find out how many `Community`, `Undergrad`, `Grad`, `Student-Other`, `Campus Affiliate`, `Rec Well Staff`, and `Day/Week Pass` unique users there are

In [ ]:

final_df.loc[final_df['Membership_L2']=='Student', 'Membership_L2'] = 'Student-Other'
fitness_group_nick=final_df[final_df['Fitness: Group Classes Nick']>0]

community=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Community'])
undergrad=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Undergrad'])
grad=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Grad'])
other_student=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Student-Other'])
campusaff=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Campus Affiliate'])
recwellstaff=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Rec Well Staff'])
daypass=len(fitness_group_nick[fitness_group_nick['Membership_L2']=='Day/Week Pass'])

total_users = pd.Series([community, undergrad, grad, other_student, campusaff, recwellstaff, daypass], 
                        index=['Community', 'Undergrad', 'Grad', 'Student-Other', 'Campus Affiliate', 'Rec Well Staff', 'Day/Week Pass'])

ax=total_users.plot.bar(color='teal')

for i in range(len(total_users)):
    ax.text(i, total_users[i]+50, str(total_users[i]), ha='center', fontsize='small')

ax.set_xlabel('User Types')
ax.set_ylabel('Number of Unique Users')
ax.set_title('User Types Within Fitness: Group Classes Nick')
